<a href="https://colab.research.google.com/github/ridvanyigit/LLMs/blob/main/PDF_RAG_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Imports

import os
from dotenv import load_dotenv

import gradio as gr
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_core.callbacks import StdOutCallbackHandler

In [ ]:
# 🔐 API Key
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [ ]:
pdf_path = "knowledge-base/your-pdf-file.pdf"
db_name = "vector_db"
model_name = "gpt-4o"

In [ ]:
# 📚 PDF yükleyici
loader = PyPDFLoader(pdf_path)
documents = loader.load()

In [ ]:
# ✂️ Chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

In [ ]:
# 🔎 Embedding + Chroma DB
embeddings = OpenAIEmbeddings()
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=db_name)
vectorstore.persist()

In [ ]:
# 🧠 Memory, LLM ve retriever setup
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
llm = ChatOpenAI(model_name=model_name, temperature=0.7)
retriever = vectorstore.as_retriever(search_kwargs={"k": 25})

# 🧵 Callback, LLM background
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    callbacks=[StdOutCallbackHandler()]
)

In [ ]:
# 💬 Gradio Chat Interface function
def chat(question, history):
    result = conversation_chain.invoke({"question": question})
    return result["answer"]

In [ ]:
# 🚀 Run UI
gr.ChatInterface(chat, type="messages").launch(inbrowser=True)